In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import openpyxl
from scipy.optimize import dual_annealing

In [2]:
# Replace 'your_excel_file.xlsx' with the actual path to your Excel file
excel_file = 'OGCS_final_parameters.xlsx'

# List of sheet names
names = [
    "T",   # Test/initial trial of T (Optimization Variable) - Average person-miles by mode,distance
    "x",    # Average per-day needs per person in effective miles traveled
    "v_max", # Maximum speed of travel allowed by each mode/distance combination
    "c",  # Effects of mode use on congestion of each other mode respectively
    "t^t", # Time - per - trip added (e.g. 'up-front' time cost of travel) per mode
    "d", # Real miles traveled per trip per mode/effective distance bucket
    "d^e", # Average effective miles per bucket (indicitave of modeling choices for bucket locations & sizes accross possible distance traveled spectrum); (0-1), (1-5), (5-20), (20-100) km
    "alpha", # Weighting constants for different components of utility function (e.g. importance of effort, time, cost, etc. related to each other)
    "a", # Constants of the system generic to all modes (Baseline electricity loads, population, effectiveness in managing congestion, etc.)
    "f", # Fuel efficiency of respective transportation type
    "nu", # Safety sensitivity to weather
    "b", # Effort component
    "e", # Electricity effieiency
    "t^b", # Time of beginning use/recharge
    "t^e", # Time of ending use/recharge
    "r", # Renewables availability (proportion)
    "l_b^t", # Current load requirements (proportion)
    "p_b^t", # Current baseline prices (proportion)
    "T_min", # Minimum Bounds on T per the optimization tool
    "T_max", # Maximum Bounds on T per the optimization tool
]

# Create a dictionary to store the dataframes
prms = {}

# Loop through the sheet names and read each sheet into a dataframe
for i, sheet_name in enumerate(names):
    try:
        # Read the sheet into a pandas DataFrame
        df = pd.read_excel(excel_file, sheet_name=i, header=0, index_col=0)  # Use sheet index instead of name

        # Assign the dataframe to a variable with the corresponding name
        prms[sheet_name] = df.values[:,:]

        print(f"Successfully read data from sheet {i+1} into DataFrame '{sheet_name}'")

    except Exception as e:
        print(f"Error reading sheet {i+1}: {e}")

#print_(prms["t^t"]) #Example of calling data from one of these variables

Successfully read data from sheet 1 into DataFrame 'T'
Successfully read data from sheet 2 into DataFrame 'x'
Successfully read data from sheet 3 into DataFrame 'v_max'
Successfully read data from sheet 4 into DataFrame 'c'
Successfully read data from sheet 5 into DataFrame 't^t'
Successfully read data from sheet 6 into DataFrame 'd'
Successfully read data from sheet 7 into DataFrame 'd^e'
Successfully read data from sheet 8 into DataFrame 'alpha'
Successfully read data from sheet 9 into DataFrame 'a'
Successfully read data from sheet 10 into DataFrame 'f'
Successfully read data from sheet 11 into DataFrame 'nu'
Successfully read data from sheet 12 into DataFrame 'b'
Successfully read data from sheet 13 into DataFrame 'e'
Successfully read data from sheet 14 into DataFrame 't^b'
Successfully read data from sheet 15 into DataFrame 't^e'
Successfully read data from sheet 16 into DataFrame 'r'
Successfully read data from sheet 17 into DataFrame 'l_b^t'
Successfully read data from sheet 18

In [3]:
sclrs = {}

scalar_names = [
    "t_max", # Max Allowable Travel Time (hrs)
    "g", # Congestion Weighting Constant (Unitless)
    "h_w", # Wind Average Constant (kW)
    "h_s", # Solar Average Constant (kW)
    "l_b", # Baseline Average Load (kW)
    "gamma", # System Energy Peaker Inefficiency/Marginal Cost Incerase ($/kW^2)
    "p_bse", # Baseline Price per kWh ($/kWh) (Going off of use of renewables)
    "i", # System population
    "w", # Weather conditions
    "lf", # Load Factor
    "rm", # Reserve Margin
    ]

for i, scalar_name in enumerate(scalar_names):
    try:
        sclrs[scalar_name] = prms["a"][i]
        print(f"Successfully assigned value to {scalar_name} as {sclrs[scalar_name]}")
    except Exception as e:
        print(f"Error reading sheet {i+1}: {e}")

Successfully assigned value to t_max as [6.]
Successfully assigned value to g as [0.001]
Successfully assigned value to h_w as [102375.]
Successfully assigned value to h_s as [43875.]
Successfully assigned value to l_b as [375000.]
Successfully assigned value to gamma as [80.]
Successfully assigned value to p_bse as [0.157]
Successfully assigned value to i as [500000.]
Successfully assigned value to w as [50.]
Successfully assigned value to lf as [0.85]
Successfully assigned value to rm as [0.15]


In [4]:
def print_(str, print_choose):
    if print_choose == 0:
        return
    else:
        print(str)
        return

def find_cost(T, prms, sclrs, print_choose):
    constraints = [0, 0, 0]

    print_(f"Checking Constraint for Effective Miles Traveled: ", print_choose)

    x_test = np.sum(T, axis=0)

    if np.all(x_test > np.squeeze(prms["x"])):
        print_("TRUE \n", print_choose)
        constraints[0] = 1
    else:
        print_("FALSE! Change T or X\n", print_choose)

    # %%
    print_(f"Average number of trips per mode, bucket", print_choose)
    trips = np.zeros(T.shape)
    for i in range (trips.shape[1]):
        trips[:,i] = T[:,i] / prms["d^e"][i]
    print_(f"{trips}\n", print_choose)

    print_("Estimating Average number of effective miles traveled per mode, bucket", print_choose)
    T_actual = np.multiply(trips, prms["d"])
    print_(f"T_actual", print_choose)

    print_("Mutual congestion effects", print_choose)
    Total_mode_actual = np.sum(T_actual, axis = 1)
    congestion_effects = np.zeros_like(Total_mode_actual)
    for i_1 in range (T_actual.shape[0]):
        c_e_counter = 0
        for i_2 in range (T_actual.shape[0]):
            c_e_counter += prms["c"][i_1, i_2] * Total_mode_actual[i_2]
        congestion_effects[i_1] = c_e_counter
    congestion_effects = congestion_effects * sclrs["g"]
    print_(congestion_effects, print_choose)

    print_("Real Velocity of Travelers", print_choose)
    v_real = prms["v_max"] * np.exp(-1 * congestion_effects.reshape(prms["v_max"].shape[0],1))
    print_(f"{v_real}\n", print_choose)

    print_("Real Travel Times", print_choose)
    t_travel = T_actual / v_real
    print_(f"{t_travel}\n", print_choose)

    print_("Real Added Wait Time", print_choose)
    t_wait = trips * prms["t^t"]
    print_(f"{t_wait}\n", print_choose)

    print_("Total Time per Mode-Bucket", print_choose)
    t_total = t_travel + t_wait
    print_(f"{t_total}\n", print_choose)

    print_(f"Total per Person Travel Time: {np.sum(t_total)} hrs", print_choose)

    print_("Checking Total Time Constraint:\n", print_choose)
    if(np.sum(t_total) > sclrs["t_max"]):
        print_(f"FALSE. Travel less, or reconsider parameters.\n", print_choose)
    else:
        print_("TRUE\n", print_choose)
        constraints[1] = 1

    # %%
    #How we understand when vehicles will be charging
    def get_time_charge(i):
        result_matrix = np.ones_like(t_b_electric)

        result_matrix[(i-t_b_electric) % 24 >= (t_e_electric - t_b_electric) % 24] = 0

        return result_matrix * load_rates

    print_("System Maximum Capacity", print_choose)
    l_max = sclrs["l_b"] * 1/sclrs["lf"] * (1 + sclrs["rm"])
    print_(f"{l_max}\n", print_choose)

    print_("Lowest Renewable Available", print_choose)
    renewable_avail = prms["r"][:,1] * sclrs["h_s"] + prms["r"][:,0] * sclrs["h_w"]
    r_min = min(renewable_avail)
    print_(f"{r_min}\n", print_choose)

    print_("Normal Dispatchable Expectation", print_choose)
    dispatch_max = l_max - r_min
    print_(f"{dispatch_max}\n", print_choose)

    print_("Electricity Expended / Person", print_choose)
    indeces_electric = [1,4]
    T_actual_electric = T_actual[indeces_electric, :]
    e_electric = prms["e"][indeces_electric, :]
    electricity_expended = T_actual_electric / e_electric
    print_(f"{electricity_expended}\n", print_choose)

    print_("kWh / hr load  / Person", print_choose)
    t_b_electric = prms["t^b"][indeces_electric,:]
    t_e_electric = prms["t^e"][indeces_electric,:]
    charge_intervals = (t_e_electric - t_b_electric) % 24
    load_rates_normal = electricity_expended / charge_intervals
    print_(f"{load_rates_normal}\n", print_choose)

    print_("kWh / hr load", print_choose)
    load_rates = load_rates_normal * sclrs["i"]
    print_(f"{load_rates}\n", print_choose)

    print_("Baseline Load (kW)", print_choose)
    baseline_load = sclrs["l_b"] * prms["l_b^t"]
    print_(f"{baseline_load}\n", print_choose)

    print_("Total Load (kW)", print_choose)
    time_block = np.zeros((t_b_electric.shape[0], t_b_electric.shape[1], 24), dtype=int)
    total_load = np.zeros_like(baseline_load)
    for k in np.arange(24):
        time_block[:, :, k] = get_time_charge(k)
        total_load[k] = baseline_load[k] + np.sum(time_block[:,:,k])
    print_(f"{total_load}\n", print_choose)

    print_("Max Load Required by Dispatch Resources (kW)", print_choose)
    max_dispatchable_load = np.max(np.squeeze(total_load) - renewable_avail)
    print_(f"{max_dispatchable_load}\n", print_choose)

    print_("Checking Max Load Constraint (kW)", print_choose)
    if(max_dispatchable_load > dispatch_max):
        print_(f"FALSE. Travel less, or reconsider parameters.\n", print_choose)
    else:
        print_("TRUE\n", print_choose)
        constraints[2] = 1





    # %% [markdown]
    # alpha dictionary:
    # 
    # 0: Trips
    # 
    # 1: Time
    # 
    # 2: Fuel
    # 
    # 3: Effort
    # 
    # 4: Electricity
    # 
    # 5: Safety

    # %%
    #Calculate Utility from Extra Trips Added:

    U = np.zeros([6])

    print_("Number of extra trips, per bucket:", print_choose)
    extra_miles = np.sum(T, axis=0) - np.squeeze(prms["x"])
    extra_trips = extra_miles / np.squeeze(prms["d^e"])
    print_(f"{extra_trips}\n", print_choose)

    print_("Utility from extra trips:", print_choose)
    U[0] = prms["alpha"][0] * np.sum(np.log(np.where(extra_trips < 0, 0, extra_trips) + 1))
    print_(U[0], print_choose)


    # %%
    #Calculate Utility from Time Spent:

    print_("Utility from Time Cost:", print_choose)
    U[1] = prms["alpha"][1] * np.sum(t_total)
    print_(U[1], print_choose)

    # %%
    #Calculate Utility from Fuel Cost:

    fuel_indeces = [0,2,3]
    f_fuel = prms["f"][fuel_indeces, :]
    T_actual_fuel = T_actual[fuel_indeces, :]

    print_("Fuel Expended: gal / Person:", print_choose)
    fuel_expended = T_actual_fuel/f_fuel
    print_(f"{fuel_expended}\n", print_choose)

    print_("Utility from Fuel:", print_choose)
    U[2] = prms["alpha"][2] * np.sum(fuel_expended)
    print_(U[2], print_choose)

    # %%
    #Calculate Utility from Effort

    print_("Utility from Effort:", print_choose)
    effort_matrix = np.multiply(trips, np.pow(prms["d"],2))
    U[3] = prms["alpha"][3] * np.sum(np.multiply(effort_matrix, prms["b"]))
    print_(f"{U[3]}\n", print_choose)

    print_(np.multiply(effort_matrix, prms["b"]), print_choose)

    # %%
    # Electricity Cost

    print_("Load Expected to be Dispatched to Thermal (kW):", print_choose)
    dispatchable_load = np.squeeze(total_load) - renewable_avail
    print_(f"{dispatchable_load}\n", print_choose)

    print_("Expected Dispatch to Thermal Load Over Minimum (kW):", print_choose)
    min_dispatch = min(dispatchable_load)
    dispatch_excess = dispatchable_load - min_dispatch
    print_(f"{dispatch_excess}\n", print_choose)

    print_("Marginal Effects on Price:", print_choose)
    min_baseline_load = min(baseline_load)
    marginal_effects = sclrs["gamma"] * np.pow(dispatch_excess / min_baseline_load, 2)
    print_(f"{marginal_effects}\n", print_choose)

    print_("Baseline Prices:", print_choose)
    print_(f"{sclrs["p_bse"] * np.squeeze(prms["p_b^t"])}\n", print_choose)

    print_("Adjusted Price for Marginal Impact:", print_choose)
    real_prices = sclrs["p_bse"] * (np.squeeze(prms["p_b^t"]) + marginal_effects)
    print_(f"{real_prices}\n", print_choose)

    print_("Utility from Cost:", print_choose)
    cost_count_electric = 0
    for i in range(24):
        cost_count_electric += np.sum(time_block[:,:,i] / sclrs["i"] * real_prices[i])
    U[4] = prms["alpha"][4] * cost_count_electric
    print_(f"{U[4]}\n", print_choose)



    #dynamic_electricity_price = sclrs["gamma"] * np.pow(dispatchable_load, 2) * 

    #print_(dynamic_electricity_price)

    # %%
    # Calculate Safety Impacts

    print_("Utility from Safety Effects:", print_choose)
    U[5] = prms["alpha"][5] * np.sum(np.multiply(prms["nu"], T_actual) * sclrs["w"])
    print_(f"{U[5]}\n", print_choose)

    # %%
    # Finally, What's the final utility!?

    print_("Final Utility Value:", print_choose)
    maximize_me = np.sum(U)
    print_(maximize_me, print_choose)

    if np.any(np.array(constraints) == 0):
        maximize_me -= 10000000

    return constraints, maximize_me



In [5]:
result = find_cost(prms["T"], prms, sclrs, 0)

print(result)

([1, 1, 1], np.float64(203.26383779711844))


C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:161: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[0] = prms["alpha"][0] * np.sum(np.log(np.where(extra_trips < 0, 0, extra_trips) + 1))
C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:169: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[1] = prms["alpha"][1] * np.sum(t_total)
C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.2

In [6]:
# Define the objective function to minimize

def objective_function(T):
    return -1 * find_cost(np.reshape(T, (7,4)), prms, sclrs, 0)[1]

 # Define the bounds of the search space
bounds = []
for i in range(7):
     for j in range(4):
         bounds.append((prms["T_min"][i, j], prms["T_max"][i, j]))

print(prms["T_max"])

  #Perform dual annealing optimization
result = dual_annealing(objective_function, bounds)

 #Print the results
print("Global minimum found at:\n", result.x)
print("Minimum value:", result.fun)


[[2.10e+00 4.20e+00 2.10e+01 6.00e+01]
 [3.00e-01 2.70e+00 4.50e+00 1.20e+01]
 [1.50e-01 6.00e-01 4.50e+00 6.00e+00]
 [2.10e-01 3.00e+00 9.00e+00 1.20e+01]
 [2.10e-01 3.00e+00 9.00e+00 1.20e+01]
 [2.70e+00 7.50e-01 3.00e-05 3.00e-06]
 [1.50e+00 2.25e+00 9.00e+00 9.00e+00]]


C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:161: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[0] = prms["alpha"][0] * np.sum(np.log(np.where(extra_trips < 0, 0, extra_trips) + 1))
C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:169: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  U[1] = prms["alpha"][1] * np.sum(t_total)
C:\Users\cjgir\AppData\Local\Temp\ipykernel_31656\2359942991.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.2

Global minimum found at:
 [4.05260166e-01 4.00039804e+00 9.87837755e+00 8.30825577e+00
 1.63799963e-01 1.40859585e+00 2.28336533e+00 1.19932117e+00
 1.44092037e-01 5.69272405e-01 3.32914738e+00 6.48280936e-04
 1.64057203e-04 4.34566123e-01 7.48612952e+00 1.87145718e+00
 1.96126188e-01 2.68933067e+00 1.09927753e+00 8.62040442e+00
 2.63926135e+00 4.92775593e-01 4.50989065e-07 3.73596486e-07
 1.49806028e+00 2.24460651e+00 8.53750628e+00 1.65809204e-05]
Minimum value: -474.7862052970566
